In [3]:
import gradio
import numpy as np
from DeepPurpose import utils, models, property_pred

def DTI_pred(drug, target, drug_encoding, target_encoding, model):
    X_pred = utils.data_process(X_drug = [drug], X_target = [target], y = [0],
                                drug_encoding = drug_encoding, target_encoding = target_encoding, 
                                split_method='no_split')
    y_pred = model.predict(X_pred)[0]
    y_pred = 10**(-y_pred) / 1e-9
    return f'{y_pred:.2f} nM'


def ADMET_pred(drug, ADMET_model_type, task, unit):
    model = model_ADMETs[task + '_' + ADMET_model_type + '_model']
    X_pred = utils.data_process(X_drug = [drug], y = [0],
                                drug_encoding = ADMET_model_type, 
                                split_method='no_split')
    y_pred = model.predict(X_pred)[0]
    
    if unit == '%':
        y_pred = y_pred*100
    
    return f'{y_pred:.2f} ' + unit

model_DTIs = {
    'CNN-CNN': models.model_pretrained(model = 'CNN_CNN_BindingDB'),
    'MPNN-CNN': models.model_pretrained(model = 'MPNN_CNN_BindingDB'),
    'Morgan-CNN': models.model_pretrained(model = 'Morgan_CNN_BindingDB'),
    'Morgan-AAC': models.model_pretrained(model = 'Morgan_AAC_BindingDB'),
    'Daylight-AAC': models.model_pretrained(model = 'Daylight_AAC_BindingDB'),
}

model_ADMETs = {}

for ADMET_model_type in ['MPNN', 'CNN', 'Morgan']:
    for task in ['AqSolDB', 'Caco2', 'HIA', 'Pgp_inhibitor', 'Bioavailability', 'BBB_MolNet', 'PPBR', 'CYP2C19', 'CYP2D6', 'CYP3A4', 'CYP1A2', 'CYP2C9', 'ClinTox', 'Lipo_AZ', 'Half_life_eDrug3D', 'Clearance_eDrug3D']:
        model_ADMETs[task + '_' + ADMET_model_type + '_model'] = property_pred.model_pretrained(model = task + '_' + ADMET_model_type + '_model')
        
def chem(amino_acid_sequence, molecule, affinity_model_type, ADMET_model_type):
    # affinity scores
    if len(amino_acid_sequence) > 0:
        # there is AMINO ACID SEQUENCE INPUT
        model_DTI = model_DTIs[affinity_model_type]
        affinity = DTI_pred(molecule, amino_acid_sequence, affinity_model_type.split('-')[0], affinity_model_type.split('-')[1], model_DTI)
    else:
        affinity = 'Missing Amino Acid Sequence, no Binding Affinity is predicted'
    
    ADMET = {}
    # physiochem
    solubility = ADMET_pred(molecule, ADMET_model_type, 'AqSolDB', 'log mol/L')
    Lipophilicity = ADMET_pred(molecule, ADMET_model_type, 'Lipo_AZ', '(log-ratio)')
    physiochem = {'Solubility': solubility, 'Lipophilicity': Lipophilicity}
    ADMET.update(physiochem)

    # absorption
    Caco2 = ADMET_pred(molecule, ADMET_model_type, 'Caco2', 'cm/s')
    HIA = ADMET_pred(molecule, ADMET_model_type, 'HIA', '%')
    Pgp = ADMET_pred(molecule, ADMET_model_type, 'Pgp_inhibitor', '%')
    bioavail = ADMET_pred(molecule, ADMET_model_type, 'Bioavailability', '%')
    
    absorption = {'(Absorption) Caco-2': Caco2, '(Absorption) HIA': HIA, '(Absorption) Pgp': Pgp, '(Absorption) Bioavailability F20': bioavail}
    ADMET.update(absorption)
    
    # distribution
    BBB = ADMET_pred(molecule, ADMET_model_type, 'BBB_MolNet', '%')
    PPBR = ADMET_pred(molecule, ADMET_model_type, 'PPBR', '%')

    distribution = {'(Distribution) BBB': BBB, '(Distribution) PPBR': PPBR}
    ADMET.update(distribution)
    
    # Metabolism
    CYP2C19 = ADMET_pred(molecule, ADMET_model_type, 'CYP2C19', '%')
    CYP2D6 = ADMET_pred(molecule, ADMET_model_type, 'CYP2D6', '%')
    CYP3A4 = ADMET_pred(molecule, ADMET_model_type, 'CYP3A4', '%')
    CYP1A2 = ADMET_pred(molecule, ADMET_model_type, 'CYP1A2', '%')
    CYP2C9 = ADMET_pred(molecule, ADMET_model_type, 'CYP2C9', '%')
    
    metabolism = {'(Metabolism) CYP2C19': CYP2C19, '(Metabolism) CYP2D6': CYP2D6, '(Metabolism) CYP3A4': CYP3A4, '(Metabolism) CYP1A2': CYP1A2, '(Metabolism) CYP2C9': CYP2C9}
    ADMET.update(metabolism)
    
    # Execretion
    Half_life = ADMET_pred(molecule, ADMET_model_type, 'Half_life_eDrug3D', 'h')
    Clearance = ADMET_pred(molecule, ADMET_model_type, 'Clearance_eDrug3D', 'mL/min/kg')
        
    execretion = {'(Execretion) Half life': Half_life, '(Execretion) Clearance': Clearance}
    ADMET.update(execretion)
    
    # Clinical Toxicity
    toxicity = ADMET_pred(molecule, ADMET_model_type, 'ClinTox', '%')
        
    clinical_toxicity = {'Clinical Toxicity': toxicity}
    ADMET.update(clinical_toxicity)
    
    return molecule, affinity, ADMET

gradio.Interface(chem, [gradio.inputs.Textbox(label = "Amino Acid Sequence", lines = 3, default = 'SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ'), 
                        gradio.inputs.Textbox(label = "SMILES", lines = 3), 
                        gradio.inputs.Dropdown(label = "Affinity Prediction Model Type", choices = ['MPNN-CNN', 'CNN-CNN', 'Morgan-CNN', 'Morgan-AAC', 'Daylight-AAC']),
                        gradio.inputs.Dropdown(label = "ADMET Prediction Model Type", choices = ['MPNN', 'Morgan', 'CNN'])], 
                       [gradio.outputs.Textbox(label = "Canonical SMILES"),
                        gradio.outputs.Textbox(label = "Binding Affinity (Kd)"),
                        gradio.outputs.KeyValues(label = "Predicted ADMET Property")], 
                        layout="vertical",
                        description="Kexin Huang, Tianfan Fu, Dawood Khan, Ali Abid, Ali Abdalla, Abubakar Abid, Lucas M. Glass, Marinka Zitnik, Cao Xiao, Jimeng Sun. \ DeepPurpose GitHub Repository: https://github.com/kexinhuang12345/DeepPurpose || Live Website: deeppurpose.sublab.org",
                        title="[NeurIPS 2020 Demo] MolDesigner: Interactive Design of Efficacious Drugs with Deep Learning, powered by DeepPurpose").launch(share = False)

Beginning Downloading CNN_CNN BindingDB Model...
pretrained model Successfully Downloaded...
Beginning Downloading MPNN_CNN_BindingDB Model...
pretrained model Successfully Downloaded...
Beginning Downloading Morgan_CNN_BindingDB Model...
pretrained model Successfully Downloaded...
Beginning Downloading Morgan_AAC_BindingDB Model...
pretrained model Successfully Downloaded...
Beginning Downloading Daylight_AAC_BindingDB Model...
pretrained model Successfully Downloaded...
Beginning DownloadingAqSolDB_MPNN_model Model...
Beginning DownloadingCaco2_MPNN_model Model...
Beginning DownloadingHIA_MPNN_model Model...
Beginning DownloadingPgp_inhibitor_MPNN_model Model...
Beginning DownloadingBioavailability_MPNN_model Model...
Beginning DownloadingBBB_MolNet_MPNN_model Model...
Beginning DownloadingPPBR_MPNN_model Model...
Beginning DownloadingCYP2C19_MPNN_model Model...
Beginning DownloadingCYP2D6_MPNN_model Model...
Beginning DownloadingCYP3A4_MPNN_model Model...
Beginning DownloadingCYP1A2

(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x7fef21357c50>,
 'http://127.0.0.1:7863/',
 None)